## ThinkDSP

This notebook contains code examples from Chapter 3: Non-periodic signals

Copyright 2015 Allen Downey

License: [Creative Commons Attribution 4.0 International](http://creativecommons.org/licenses/by/4.0/)

In [ ]:
from __future__ import print_function, division

%matplotlib inline

import thinkdsp
import thinkplot
import numpy as np

from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

### Chirp

Make a linear chirp from A3 to A5.

In [ ]:
signal = thinkdsp.Chirp(start=220, end=880)
wave1 = signal.make_wave(duration=2)
wave1.make_audio()

Here's what the waveform looks like near the beginning.

In [ ]:
wave1.segment(start=0, duration=0.01).plot()

And near the end.

In [ ]:
wave1.segment(start=0.9, duration=0.01).plot()

Here's an exponential chirp with the same frequency range and duration.

In [ ]:
signal = thinkdsp.ExpoChirp(start=220, end=880)
wave2 = signal.make_wave(duration=2)
wave2.make_audio()

In [ ]:
spectrum = wave1.make_spectrum()
spectrum.plot()

In [ ]:
spectrum = wave2.make_spectrum()
spectrum.plot()

## Leakage

Spectral leakage is when some of the energy at one frequency appears at another frequency (usually nearby).

Let's look at the effect of leakage on a sine signal (which only contains one frequency component).

In [ ]:
signal = thinkdsp.SinSignal(freq=440)

If the duration is an integer multiple of the period, the beginning and end of the segment line up, and we get minimal leakage.

In [ ]:
duration = signal.period * 30
wave = signal.make_wave(duration)
wave.plot()

In [ ]:
spectrum = wave.make_spectrum()
spectrum.plot(high=880)
thinkplot.config(xlabel='frequency (Hz)', ylabel='amplitude', legend=False)

If the duration is not a multiple of a period, the leakage is pretty bad.

In [ ]:
duration = signal.period * 30.25
wave = signal.make_wave(duration)
wave.plot()

In [ ]:
spectrum = wave.make_spectrum()
spectrum.plot(high=880)
thinkplot.config(xlabel='Frequency (Hz)')

Windowing helps (but notice that it reduces the total energy).

In [ ]:
wave.hamming()
spectrum = wave.make_spectrum()
spectrum.plot(high=880)
thinkplot.config(xlabel='Frequency (Hz)')

## Spectrogram

If you blindly compute the DFT of a non-periodic segment, you get "motion blur".

In [ ]:
signal = thinkdsp.Chirp(start=220, end=440)
wave = signal.make_wave(duration=1)
spectrum = wave.make_spectrum()
spectrum.plot(high=700)
thinkplot.config(xlabel='frequency (Hz)')

A spectrogram is a visualization of a short-time DFT that lets you see how the spectrum varies over time.

In [ ]:
def plot_spectrogram(wave, seg_length):
    spectrogram = wave.make_spectrogram(seg_length)
    print('Time resolution (s)', spectrogram.time_res)
    print('Frequency resolution (Hz)', spectrogram.freq_res)
    spectrogram.plot(high=700)
    thinkplot.show(xlabel='Time(s)', ylabel='Frequency (Hz)')

In [ ]:
signal = thinkdsp.Chirp(start=220, end=440)
wave = signal.make_wave(duration=1, framerate=11025)
plot_spectrogram(wave, 512)

If you increase the segment length, you get better frequency resolution, worse time resolution.

In [ ]:
plot_spectrogram(wave, 1024)

If you decrease the segment length, you get better time resolution, worse frequency resolution.

In [ ]:
plot_spectrogram(wave, 256)

In [ ]:
slider = widgets.IntSlider(min=128, max=4096, value=100, step=128)
interact(plot_spectrogram, wave=fixed(wave), seg_length=slider);

## Spectrum of a chirp

The following interaction lets you customize the Eye of Sauron as you vary the start and end frequency of the chirp.

In [ ]:
def eye_of_sauron(start, end):
    """Plots the spectrum of a chirp.
    
    start: initial frequency
    end: final frequency
    """
    signal = thinkdsp.Chirp(start=start, end=end)
    wave = signal.make_wave(duration=0.5)
    spectrum = wave.make_spectrum()
    
    spectrum.plot(high=1200)
    thinkplot.config(xlabel='frequency (Hz)', ylabel='amplitude')
    
    #TODO: add player

In [ ]:
slider1 = widgets.FloatSlider(min=100, max=1000, value=100, step=50)
slider2 = widgets.FloatSlider(min=100, max=1000, value=200, step=50)
interact(eye_of_sauron, start=slider1, end=slider2);

# Excercises 

## 3.2

Write a class called SawtoothChirp that extends Chirp and
overrides evaluate to generate a sawtooth waveform with frequency that
increases (or decreases) linearly.

Hint: combine the evaluate functions from Chirp and SawtoothSignal.

Draw a sketch of what you think the spectrogram of this signal looks like,
and then plot it. The effect of aliasing should be visually apparent, and if
you listen carefully, you can hear it.


In [ ]:
class SawtoothChirp(thinkdsp.Chirp):
    """Represents a sawtooth signal with varying frequency."""

    def _evaluate(self, ts, freqs):
        """Helper function that evaluates the signal.

        ts: float array of times
        freqs: float array of frequencies during each interval
        """
        
        """
        # Chirp
        start, end = np.log10(self.start), np.log10(self.end)
        freqs = np.logspace(start, end, len(ts) - 1)
        return self._evaluate(ts, freqs)
        """

        """
        #Sawtooth
        ts = np.asarray(ts)
        cycles = self.freq * ts + self.offset / PI2
        frac, _ = np.modf(cycles)
        ys = normalize(unbias(frac), self.amp)
        return ys
        """
        
        dts = np.diff(ts)
        dps = PI2 * freqs * dts
        phases = np.cumsum(dps)
        phases = np.insert(phases, 0, 0)
        cycles = phases / PI2
        frac, _ = np.modf(cycles)
        ys = thinkdsp.normalize(thinkdsp.unbias(frac), self.amp)
        return ys
        

In [ ]:
import math
PI2 = 2 * math.pi
#
signal = SawtoothChirp(start=220, end=880)
wave = signal.make_wave(duration=1, framerate=10000)
wave.apodize()
wave.make_audio()

In [ ]:
wave.segment(start=0, duration=0.1).plot()

In [ ]:
spectrum = wave.make_spectrum()
spectrum.plot()

In [ ]:
sp = wave.make_spectrogram(1024)
sp.plot()
thinkplot.config(xlabel='Time (s)', ylabel='Frequency (Hz)')

## 3.3

Make a sawtooth chirp that sweeps from 2500 to 3000 Hz, then
use it to make a wave with duration 1 s and framerate 20 kHz. Draw a
sketch of what you think the spectrum will look like. Then plot the spectrum and see if you got it right.

In [ ]:
signal = SawtoothChirp(start=2500, end=3000)
wave = signal.make_wave(duration=1, framerate=20000)
wave.apodize()
wave.make_audio()

In [ ]:
spectrum = wave.make_spectrum()
spectrum.plot()

## 3.4

 In musical terminology, a “glissando” is a note that slides from
one pitch to another, so it is similar to a chirp.
Find or make a recording of a glissando and plot a spectrogram of the first
few seconds. One suggestion: George Gershwin’s Rhapsody in Blue starts
with a famous clarinet glissando, which you can download from http://
archive.org/details/rhapblue11924.

In [ ]:
wave = thinkdsp.read_wave('72475__rockwehrmann__glissup02.wav')
wave.make_audio() 

In [ ]:
wave.make_spectrogram(512).plot(high=5000)

## 3.5

A trombone player can play a glissando by extending the trombone slide while blowing continuously. As the slide extends, the total length
of the tube gets longer, and the resulting pitch is inversely proportional to
length.

Assuming that the player moves the slide at a constant speed, how does
frequency vary with time?

Write a class called TromboneGliss that extends Chirp and provides
evaluate. Make a wave that simulates a trombone glissando from C3 up
to F3 and back down to C3. C3 is 262 Hz; F3 is 349 Hz.

Plot a spectrogram of the resulting wave. Is a trombone glissando more like
a linear or exponential chirp?


In [ ]:
class TromboneGliss(thinkdsp.Chirp):
    """Represents a trombone-like signal with varying frequency."""
    
    def evaluate(self, ts):
        """Evaluates the signal at the given times.

        ts: float array of times
        
        returns: float wave array
        """
        l1, l2 = 1.0 / self.start, 1.0 / self.end
        lengths = np.linspace(l1, l2, len(ts)-1)
        freqs = 1 / lengths
        return self._evaluate(ts, freqs)

In [ ]:
#1st part
low = 262
high = 349
signal = TromboneGliss(high, low)
wave1 = signal.make_wave(duration=1)
wave1.apodize()
wave1.make_audio()

In [ ]:
#2nd part
signal = TromboneGliss(low, high)
wave2 = signal.make_wave(duration=1)
wave2.apodize()
wave2.make_audio()

In [ ]:
#Together
wave = wave1 | wave2
wave.make_audio()

In [ ]:
sp = wave.make_spectrogram(2048)
sp.plot(high=1000)


## 3.6

Make or find a recording of a series of vowel sounds and look
at the spectrogram. Can you identify different vowels?

In [ ]:
wave = thinkdsp.read_wave('87778__marcgascon7__vocals.wav')
wave.make_audio()

In [ ]:
wave.make_spectrogram(1024).plot(high=1000)

The stripe across the bottom is probably background noise.  The peaks in the spectrogram are called "formants".

In general, vowel sounds are distinguished by the amplitude ratios of the first two formants relative to the fundamental.  For more, see https://en.wikipedia.org/wiki/Formant

We can see the formats more clearly by selecting a segment during 'ah'.

In [ ]:
high = 1000
thinkplot.preplot(5)

segment = wave.segment(start=1, duration=0.25)
segment.make_spectrum().plot(high=high)

The fundamental is near 100 Hz.  The next highest peaks are at 200 Hz and 700 Hz.  People who know more about this than I do can identify vowels by looking at spectrums, but I can't.

In [ ]:
segment.make_spectrogram(1024).plot(high=1000)

The 'eh' segment has a high-amplitude formant near 500 Hz.

In [ ]:
segment = wave.segment(start=2.2, duration=0.25)
segment.make_spectrum().plot(high=high)

In [ ]:
segment.make_spectrogram(1024).plot(high=1000)

The 'ih' segment has no high frequency components.

In [ ]:
segment = wave.segment(start=3.5, duration=0.25)
segment.make_spectrum().plot(high=high)

In [ ]:
segment.make_spectrogram(1024).plot(high=1000)

The 'oh' segment has a high-amplitude formant near 500 Hz, even higher than the fundamental.

In [ ]:
segment = wave.segment(start=5.1, duration=0.25)
segment.make_spectrum().plot(high=high)

In [ ]:
segment.make_spectrogram(1024).plot(high=1000)

The 'oo' segment has a high-amplitude formant near 300 Hz and no high-frequency components

In [ ]:
segment = wave.segment(start=6.5, duration=0.25)
segment.make_spectrum().plot(high=high)

In [ ]:
segment.make_spectrogram(1024).plot(high=1000)